In [253]:
import pandas as pd

In [254]:
df = pd.read_csv('train.csv' , engine = 'python' , error_bad_lines=False)



<ipython-input-254-d987533e48a1>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('train.csv' , engine = 'python' , error_bad_lines=False)
Skipping line 2419: unexpected end of data


In [255]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [256]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [257]:
df = pd.get_dummies(df, columns=['sentiment'], drop_first=True)


In [258]:
df.head()

,review,sentiment_positive
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [259]:
df = df.dropna()

In [260]:
df.shape

(2417, 2)

In [261]:
X = df['review']

In [262]:
y = df['sentiment_positive']

In [263]:
X.shape

(2417,)

In [264]:
y.shape

(2417,)

In [265]:
import tensorflow as tf

In [266]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [318]:
vocab_size = 75000

In [319]:
X.head(10)

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
5    Probably my all-time favorite movie, a story o...
6    I sure would like to see a resurrection of a u...
7    This show was an amazing, fresh & innovative i...
8    Encouraged by the positive comments about this...
9    If you like original gut wrenching laughter yo...
Name: review, dtype: object

In [320]:
import nltk
import re
from nltk.corpus import stopwords

In [321]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [322]:
from nltk.stem.porter import PorterStemmer

In [323]:
messages = X.copy()

In [324]:
messages[6]

"I sure would like to see a resurrection of a up dated Seahunt series with the tech they have today it would bring back the kid excitement in me.I grew up on black and white TV and Seahunt with Gunsmoke were my hero's every week.You have my vote for a comeback of a new sea hunt.We need a change of pace in TV and this would work for a world of under water adventure.Oh by the way thank you for an outlet like this to view many viewpoints about TV and the many movies.So any ole way I believe I've got what I wanna say.Would be nice to read some more plus points about sea hunt.If my rhymes would be 10 lines would you let me submit,or leave me out to be in doubt and have me to quit,If this is so then I must go so lets do it."

In [325]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages[i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)



In [326]:
# corpus




In [327]:
onehot_repr = [one_hot(word , vocab_size) for word in corpus]
#onehot_repr

In [328]:
print(corpus[1])
print(onehot_repr[1])

wonder littl product br br film techniqu unassum old time bbc fashion give comfort sometim discomfort sens realism entir piec br br actor extrem well chosen michael sheen got polari voic pat truli see seamless edit guid refer william diari entri well worth watch terrificli written perform piec master product one great master comedi life br br realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear play knowledg sens particularli scene concern orton halliwel set particularli flat halliwel mural decor everi surfac terribl well done
[66139, 4116, 48642, 21501, 21501, 56095, 26990, 54247, 24960, 10822, 53841, 49709, 27718, 68130, 49577, 8694, 38169, 62644, 71308, 27384, 21501, 21501, 6766, 72033, 20794, 33081, 74433, 7488, 3188, 38704, 51410, 63686, 26290, 60409, 22439, 11779, 44303, 27277, 7942, 69817, 63524, 20794, 72326, 17707, 18723, 11218, 50103, 27384, 65899, 48642, 45346, 47183, 65899, 35664, 52762, 21501, 21501, 62644, 4833, 39316,

In [329]:
sent_length = 200
padded_repr = pad_sequences(onehot_repr , padding = 'post' , maxlen = sent_length)

In [330]:
padded_repr

array([[45346, 14975, 26256, ...,     0,     0,     0],
       [66139,  4116, 48642, ...,     0,     0,     0],
       [65640, 66139, 67132, ...,     0,     0,     0],
       ...,
       [53409, 32718, 26375, ...,     0,     0,     0],
       [33608,  5557, 43455, ...,     0,     0,     0],
       [40123, 21020, 17511, ...,     0,     0,     0]], dtype=int32)

In [331]:
vector_repr = 150
model = Sequential()
model.add(Embedding(vocab_size , vector_repr , input_length = sent_length))
model.add(LSTM(200))
model.add(Dense(1 , activation ='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 200, 150)          11250000  
                                                                 
 lstm_6 (LSTM)               (None, 200)               280800    
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 11531001 (43.99 MB)
Trainable params: 11531001 (43.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [332]:
len(padded_repr),y.shape

(2417, (2417,))

In [333]:
import numpy as np
X_final = np.array(padded_repr)
y_final = np.array(y)

In [334]:
X_final.shape , y_final.shape

((2417, 200), (2417,))

In [337]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=32)

Epoch 1/20
61/61 [==============================] - 62s 1s/step - loss: 0.6893 - accuracy: 0.5613 - val_loss: 0.6868 - val_accuracy: 0.5188
Epoch 2/20
61/61 [==============================] - 60s 984ms/step - loss: 0.6483 - accuracy: 0.5706 - val_loss: 1.5570 - val_accuracy: 0.5977
Epoch 3/20
61/61 [==============================] - 59s 971ms/step - loss: 0.6199 - accuracy: 0.5923 - val_loss: 0.6733 - val_accuracy: 0.5276
Epoch 4/20
61/61 [==============================] - 66s 1s/step - loss: 0.5910 - accuracy: 0.5908 - val_loss: 0.6831 - val_accuracy: 0.5238
Epoch 5/20
61/61 [==============================] - 60s 992ms/step - loss: 0.6180 - accuracy: 0.5768 - val_loss: 0.6603 - val_accuracy: 0.5276
Epoch 6/20
61/61 [==============================] - 55s 904ms/step - loss: 0.5917 - accuracy: 0.5654 - val_loss: 0.6633 - val_accuracy: 0.5251
Epoch 7/20
61/61 [==============================] - 52s 851ms/step - loss: 0.5789 - accuracy: 0.6063 - val_loss: 0.7155 - val_accuracy: 0.5263
Epoch